# Northwind Trades ® - Enhancing Product Recommendations and Warehouse Efficiency through Data-Driven Analytics
## Part 5: Neo4j Triangle Algorithm

### Included Modules, Packages, and Supporting Functions

In [1]:
import csv
import pandas as pd

import psycopg2
from IPython.display import display
import neo4j
from neo4j import GraphDatabase

### Connect to PostgreSQL

In [3]:
connection = psycopg2.connect(
    user="postgres",
    password="ucb",
    host="postgres",
    port="5432",
    database="postgres"
)
cursor = connection.cursor()

### Connect to Neo4j

In [4]:
driver = GraphDatabase.driver("neo4j://neo4j:7687", auth=("neo4j", "ucb_mids_w205"))
session = driver.session(database="neo4j")

# Wipe out existing Neo4j database
def my_neo4j_wipe_out_database():
    "Wipe out database by deleting all nodes and relationships"
    session.run("MATCH (n)-[r]->() DELETE r")
    session.run("MATCH (n) DELETE n")

my_neo4j_wipe_out_database()

### Create product nodes

In [5]:
cursor.execute("SELECT product_id, product_name FROM products")
products = cursor.fetchall()
products_data = [{"id": row[0], "name": row[1]} for row in products]

query_create_nodes = """
UNWIND $products AS product
CREATE (:Product {product_id: product.id, name: product.name});
"""
session.run(query_create_nodes, products=products_data)

### Generate co-purchase pairs

In [6]:
co_purchase_sql = """
SELECT od1.product_id, od2.product_id
FROM order_details od1
JOIN order_details od2
  ON od1.order_id = od2.order_id AND od1.product_id < od2.product_id
"""
cursor.execute(co_purchase_sql)
co_purchase_rows = cursor.fetchall()
co_purchase_pairs = [{"pair": [row[0], row[1]]} for row in co_purchase_rows]

# Insert pairs into Neo4j
query_create_relationships = """
UNWIND $pairs AS row
MATCH (p1:Product {product_id: row.pair[0]})
MATCH (p2:Product {product_id: row.pair[1]})
MERGE (p1)-[:CO_PURCHASED]->(p2)
"""
session.run(query_create_relationships, pairs=co_purchase_pairs)

### Create graph projection

In [7]:
# Drop existing graph projection, if any
session.run("CALL gds.graph.drop('product_graph', false) YIELD graphName")

# Create new projection
session.run("""
CALL gds.graph.project(
    'product_graph',
    'Product',
    {
        CO_PURCHASED: {
            type: 'CO_PURCHASED',
            orientation: 'UNDIRECTED'
        }
    }
)
""")

### Implement triangle count algorithm

In [8]:
query_triangle_count_write = """
CALL gds.triangleCount.write('product_graph', {
  writeProperty: 'triangleCount'
})
YIELD nodePropertiesWritten
"""
session.run(query_triangle_count_write)

# Optional: Also fetch triangle counts into Python for name lookups
query_triangle_count_stream = """
CALL gds.triangleCount.stream('product_graph')
YIELD nodeId, triangleCount
RETURN gds.util.asNode(nodeId).product_id AS product_id,
       gds.util.asNode(nodeId).name AS product_name,
       triangleCount
ORDER BY triangleCount DESC, product_name
"""
result = session.run(query_triangle_count_stream)

triangle_counts = pd.DataFrame([
    {
        "product_id": row["product_id"],
        "product_name": row["product_name"],
        "triangle_count": row["triangleCount"]
    }
    for row in result
])

print(triangle_counts)

    product_id                     product_name  triangle_count
0           60                Camembert Pierrot            1074
1           41  Jack's New England Clam Chowder            1050
2            2                            Chang            1039
3           77  Original Frankfurter grüne Soße            1019
4           13                            Konbu            1001
..         ...                              ...             ...
72          37                       Gravad lax              83
73           5           Chef Anton's Gumbo Mix              76
74          15                     Genen Shouyu              49
75           9                  Mishi Kobe Niku              38
76          48                        Chocolade              22

[77 rows x 3 columns]


### Recommender System
The following code implements a digital shopping cart with a simplified recommender system. The recommender system uses the Neo4j co-purchase edges to recommend highly co-purchased products. When the user is finished, a receipt is printed with the total cost.

In [9]:
connection.rollback()

cursor.execute("""
    SELECT product_id, ROUND(AVG(unit_price::numeric), 2) AS avg_price
    FROM order_details
    GROUP BY product_id
""")
price_lookup = dict(cursor.fetchall())

# Get product name from triangle df
def get_product_names(cart_ids):
    return list(
        triangle_counts[triangle_counts["product_id"].isin(cart_ids)]["product_name"]
    )

# List all Northwinds products and product ids
def list_products():
    print("\n📦 Available Products:")
    query = "MATCH (p:Product) RETURN p.product_id AS id, p.name AS name ORDER BY id"
    for row in session.run(query):
        print(f"{row['id']}: {row['name']}")

# Compute total
def cart_total(cart):
    return round(sum(price_lookup[item] for item in cart), 2)

# Display receipt with final selection and total price
def receipt(cart):
    print("\n🧾 Final Cart:")
    print("-" * 40)
    print(f"{'Item':30} {'Price':>8}")
    print("-" * 40)
    for item in cart:
        name = get_product_names([item])[0]
        price = price_lookup[item]
        print(f"{name.ljust(30)} {f'${price:.2f}':>8}")
    print("-" * 40)
    print(f"{'TOTAL':30} {f'${cart_total(cart):.2f}':>8}")
    print("✅ Thanks for shopping with us!\n")

# Finds all co-purchased products to products in cart
# Recommends products with highest triangle counts
def recommend_products(cart):
    if not cart:
        return

    print("\n🧠 Based on your cart, you might also like:")

    query = """
    MATCH (p:Product)-[:CO_PURCHASED]->(rec:Product)
    WHERE p.product_id IN $cart AND NOT rec.product_id IN $cart
    RETURN rec.product_id AS id, rec.name AS name, rec.triangleCount AS tc
    ORDER BY tc DESC, name
    LIMIT 3
    """
    results = session.run(query, cart=cart)
    for row in results:
        print(f"{row['id']}: {row['name']} (Triangle Count: {row['tc']})")

def main():
    cart = []
    print("🛍️ Welcome to the Northwind Smart Cart!")

    while True:
        list_products()

        if cart:
            print("\n🛒 Your cart:")
            print("-" * 40)
            print(f"{'Item':30} {'Price':>8}")
            print("-" * 40)
            for item in cart:
                name = get_product_names([item])[0]
                price = price_lookup[item]
                print(f"{name.ljust(30)} {f'${price:.2f}':>8}")
            print("-" * 40)
            print(f"{'Cart Total':30} {f'${cart_total(cart):.2f}':>8}")
        else:
            print("\n🛒 Your cart is empty.")

        if cart:
            recommend_products(cart)

        choice = input("\nEnter product ID to add to cart (or 'done' to checkout): ").strip()
        if choice.lower() == "done":
            break
        elif int(choice) in range(1,78):
            cart.append(int(choice))
        else:
            print("❌ Invalid input. Please enter a valid number or 'done'.")

    receipt(cart)

main()

🛍️ Welcome to the Northwind Smart Cart!

📦 Available Products:
1: Chai
2: Chang
3: Aniseed Syrup
4: Chef Anton's Cajun Seasoning
5: Chef Anton's Gumbo Mix
6: Grandma's Boysenberry Spread
7: Uncle Bob's Organic Dried Pears
8: Northwoods Cranberry Sauce
9: Mishi Kobe Niku
10: Ikura
11: Queso Cabrales
12: Queso Manchego La Pastora
13: Konbu
14: Tofu
15: Genen Shouyu
16: Pavlova
17: Alice Mutton
18: Carnarvon Tigers
19: Teatime Chocolate Biscuits
20: Sir Rodney's Marmalade
21: Sir Rodney's Scones
22: Gustaf's Knäckebröd
23: Tunnbröd
24: Guaraná Fantástica
25: NuNuCa Nuß-Nougat-Creme
26: Gumbär Gummibärchen
27: Schoggi Schokolade
28: Rössle Sauerkraut
29: Thüringer Rostbratwurst
30: Nord-Ost Matjeshering
31: Gorgonzola Telino
32: Mascarpone Fabioli
33: Geitost
34: Sasquatch Ale
35: Steeleye Stout
36: Inlagd Sill
37: Gravad lax
38: Côte de Blaye
39: Chartreuse verte
40: Boston Crab Meat
41: Jack's New England Clam Chowder
42: Singaporean Hokkien Fried Mee
43: Ipoh Coffee
44: Gula Malacca
45:

In [10]:
session.close()
driver.close()